<a href="https://colab.research.google.com/github/maikenberthelsen/Proj1/blob/master/unetupdated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from __future__ import print_function


from google.colab import drive
drive.mount('/content/drive/')

import gzip
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import glob
import shutil
import sys
import urllib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
from mask_to_submission import *
from helpers import *
from F1_metrics import *
#from Unet import *
from image_processing import *
from image_augmentation import *
#from data_context import *
from data_extraction import *
from prediction import *
#from keras_pred import *
from unet_pred import *
from unetModel import *

import code
import tensorflow.python.platform

import numpy as np

import tensorflow as tf
from scipy import misc, ndimage
import shutil

import keras
#from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras import optimizers
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


from pathlib import Path
from sklearn.utils import class_weight, shuffle


NUM_CHANNELS = 3 # RGB images
PIXEL_DEPTH = 255
NUM_LABELS = 2
TRAINING_SIZE = 100
TESTING_SIZE = 50
VALIDATION_SIZE = 0  # Size of the validation set.
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 16 # 64
NUM_EPOCHS = 5
RESTORE_MODEL = False # If True, restore existing model instead of training a new one
RECORDING_STEP = 1000
MAX_AUG = 10
NEW_DIM_TRAIN = 400

# The size of the patches each image is split into. Should be a multiple of 4, and the image
# size would be a multiple of this. For this assignment to get the delivery correct it has to be 16
IMG_PATCH_SIZE = 16


# Extract data into numpy arrays, divided into patches of 16x16
data_dir = '/content/drive/My Drive/data/'
train_data_filename = data_dir + 'training/images/'
train_labels_filename = data_dir + 'training/groundtruth/' 
test_data_filename = data_dir + 'test_set_images'

# Directive for storing the augmented training images
imgDir = 'training/augmented/images'
groundTruthDir = 'training/augmented/groundtruth'



#earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.1)






Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [26]:
#x_train, y_train, x_test = load_data_img(train_data_filename, train_labels_filename, test_data_filename, TRAINING_SIZE, TESTING_SIZE, NEW_DIM_TRAIN)
#x_train, y_train, x_test = load_data_unet(train_data_filename, train_labels_filename, test_data_filename, TRAINING_SIZE, TESTING_SIZE, NEW_DIM_TRAIN)

x_train, y_train, x_test, x_val, y_val = load_data_unet(train_data_filename, train_labels_filename, test_data_filename, TRAINING_SIZE, TESTING_SIZE,VALIDATION_SIZE, NEW_DIM_TRAIN,
  saltpepper = 0.05,augment=True, MAX_AUG=MAX_AUG, augImgDir=imgDir , data_dir=data_dir, groundTruthDir =groundTruthDir)
print(y_train.shape)
print(x_train.shape)

Augmenting training images...
Directory  training/augmented/images  already exists, overwritten
Directory  training/augmented/groundtruth  already exists, overwritten
Test data shape:  (50, 608, 608, 3)
Number of samples in class 1 (background):  135961162
Number of samples in class 2 (road):  39878838 

(1099, 400, 400, 2)
(1099, 400, 400, 3)


In [27]:
# Class weigths
#classes = np.array([0,1])
#class_weights = class_weight.compute_class_weight('balanced',classes,y_train[:,1])
yweight = y_train[:,:,:,0]
yweight = yweight.flatten()
print(np.unique(yweight), sum(yweight))
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(yweight),
                                                 yweight)

print('Class weights: ',class_weights) 

[0. 1.] 39878838.0
Class weights:  [0.64665526 2.20467808]


In [28]:
#model = ZF_UNET_224(class_weights,NEW_DIM_TRAIN)
inputs = Input((NEW_DIM_TRAIN, NEW_DIM_TRAIN,INPUT_CHANNELS))
model = create_model(inputs,n_filters=16, dropout=0.05)
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 400, 400, 3)  0                                            
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 400, 400, 16) 448         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_19 (BatchNo (None, 400, 400, 16) 64          conv2d_20[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_19 (LeakyReLU)      (None, 400, 400, 16) 0           batch_normalization_19[0][0]     
__________________________________________________________________________________________________
conv2d_21 

In [0]:

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [0]:
# Checkpoint
filepath="/content/drive/My Drive/weightsUnet/weights.best.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#callbacks_list = [checkpoint]

callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True,mode='max')
]

In [31]:
# Train the model
print("X", x_train.shape, "y", y_train.shape)
model.fit(x_train, y_train,
          #validation_data=(x_val, y_val),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          shuffle = True,
          verbose=1,
          validation_split = 0.1,
          callbacks = callbacks,
          class_weight = class_weights
          )

X (1099, 400, 400, 3) y (1099, 400, 400, 2)
Train on 989 samples, validate on 110 samples
Epoch 1/5
989/989 [==============================] - 86s 87ms/step - loss: 0.4629 - acc: 0.7812 - val_loss: 1.4148 - val_acc: 0.7267

Epoch 00001: val_acc improved from -inf to 0.72667, saving model to /content/drive/My Drive/weightsUnet/weights.best.hdf5
Epoch 2/5
989/989 [==============================] - 78s 79ms/step - loss: 0.3529 - acc: 0.8391 - val_loss: 0.6393 - val_acc: 0.7792

Epoch 00002: val_acc improved from 0.72667 to 0.77925, saving model to /content/drive/My Drive/weightsUnet/weights.best.hdf5
Epoch 3/5
989/989 [==============================] - 78s 78ms/step - loss: 0.2976 - acc: 0.8686 - val_loss: 0.3965 - val_acc: 0.8577

Epoch 00003: val_acc improved from 0.77925 to 0.85771, saving model to /content/drive/My Drive/weightsUnet/weights.best.hdf5
Epoch 4/5
989/989 [==============================] - 78s 78ms/step - loss: 0.2644 - acc: 0.8846 - val_loss: 0.3014 - val_acc: 0.8720

Ep

In [32]:
print("X", x_train.shape, "y", y_train.shape)
model.fit(x_train, y_train,
          #validation_data=(x_val, y_val),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          shuffle = True,
          verbose=1,
          validation_split = 0.1,
          callbacks = callbacks,
          class_weight = class_weights
          )

X (1099, 400, 400, 3) y (1099, 400, 400, 2)
Train on 989 samples, validate on 110 samples
Epoch 1/5
989/989 [==============================] - 78s 78ms/step - loss: 0.2271 - acc: 0.9029 - val_loss: 0.3649 - val_acc: 0.8710

Epoch 00001: val_acc did not improve from 0.88844
Epoch 2/5
989/989 [==============================] - 78s 78ms/step - loss: 0.2079 - acc: 0.9125 - val_loss: 0.2207 - val_acc: 0.9076

Epoch 00002: val_acc improved from 0.88844 to 0.90759, saving model to /content/drive/My Drive/weightsUnet/weights.best.hdf5
Epoch 3/5
989/989 [==============================] - 78s 78ms/step - loss: 0.2013 - acc: 0.9151 - val_loss: 0.2199 - val_acc: 0.9118

Epoch 00003: val_acc improved from 0.90759 to 0.91178, saving model to /content/drive/My Drive/weightsUnet/weights.best.hdf5
Epoch 4/5
989/989 [==============================] - 77s 78ms/step - loss: 0.1899 - acc: 0.9200 - val_loss: 0.2309 - val_acc: 0.9061

Epoch 00004: val_acc did not improve from 0.91178
Epoch 5/5
989/989 [=====

In [33]:
print("X", x_train.shape, "y", y_train.shape)
model.fit(x_train, y_train,
          #validation_data=(x_val, y_val),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          shuffle = True,
          verbose=1,
          validation_split = 0.1,
          callbacks = callbacks,
          class_weight = class_weights
          )

X (1099, 400, 400, 3) y (1099, 400, 400, 2)
Train on 989 samples, validate on 110 samples
Epoch 1/5
989/989 [==============================] - 78s 78ms/step - loss: 0.1718 - acc: 0.9284 - val_loss: 0.1886 - val_acc: 0.9227

Epoch 00001: val_acc improved from 0.91178 to 0.92274, saving model to /content/drive/My Drive/weightsUnet/weights.best.hdf5
Epoch 2/5
989/989 [==============================] - 78s 78ms/step - loss: 0.1688 - acc: 0.9289 - val_loss: 0.1940 - val_acc: 0.9210

Epoch 00002: val_acc did not improve from 0.92274
Epoch 3/5
989/989 [==============================] - 78s 78ms/step - loss: 0.1604 - acc: 0.9332 - val_loss: 0.2160 - val_acc: 0.9169

Epoch 00003: val_acc did not improve from 0.92274
Epoch 4/5
989/989 [==============================] - 78s 78ms/step - loss: 0.1545 - acc: 0.9358 - val_loss: 0.1983 - val_acc: 0.9188

Epoch 00004: val_acc did not improve from 0.92274
Epoch 5/5
989/989 [==============================] - 77s 78ms/step - loss: 0.1518 - acc: 0.9365 - v

In [34]:
print("X", x_train.shape, "y", y_train.shape)
model.fit(x_train, y_train,
          #validation_data=(x_val, y_val),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          shuffle = True,
          verbose=1,
          validation_split = 0.1,
          callbacks = callbacks,
          class_weight = class_weights
          )

X (1099, 400, 400, 3) y (1099, 400, 400, 2)
Train on 989 samples, validate on 110 samples
Epoch 1/5
989/989 [==============================] - 78s 78ms/step - loss: 0.1471 - acc: 0.9386 - val_loss: 0.1971 - val_acc: 0.9259

Epoch 00001: val_acc did not improve from 0.92615
Epoch 2/5
989/989 [==============================] - 77s 78ms/step - loss: 0.1417 - acc: 0.9408 - val_loss: 0.1801 - val_acc: 0.9253

Epoch 00002: val_acc did not improve from 0.92615
Epoch 3/5
989/989 [==============================] - 77s 78ms/step - loss: 0.1406 - acc: 0.9410 - val_loss: 0.1704 - val_acc: 0.9319

Epoch 00003: val_acc improved from 0.92615 to 0.93185, saving model to /content/drive/My Drive/weightsUnet/weights.best.hdf5
Epoch 4/5
989/989 [==============================] - 77s 78ms/step - loss: 0.1282 - acc: 0.9465 - val_loss: 0.1728 - val_acc: 0.9296

Epoch 00004: val_acc did not improve from 0.93185
Epoch 5/5
989/989 [==============================] - 78s 78ms/step - loss: 0.1273 - acc: 0.9466 - v

In [35]:
'''print("X", x_train.shape, "y", y_train.shape)
model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          shuffle = True,
          verbose=1,
          #validation_split = 0.1,
          callbacks = callbacks,
          class_weight = class_weights
          )'''

'print("X", x_train.shape, "y", y_train.shape)\nmodel.fit(x_train, y_train,\n          validation_data=(x_val, y_val),\n          batch_size=BATCH_SIZE,\n          epochs=NUM_EPOCHS,\n          shuffle = True,\n          verbose=1,\n          #validation_split = 0.1,\n          callbacks = callbacks,\n          class_weight = class_weights\n          )'

In [36]:
'''print("X", x_train.shape, "y", y_train.shape)
model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          shuffle = True,
          verbose=1,
          #validation_split = 0.1,
          callbacks = callbacks,
          class_weight = class_weights
          )'''

'print("X", x_train.shape, "y", y_train.shape)\nmodel.fit(x_train, y_train,\n          validation_data=(x_val, y_val),\n          batch_size=BATCH_SIZE,\n          epochs=NUM_EPOCHS,\n          shuffle = True,\n          verbose=1,\n          #validation_split = 0.1,\n          callbacks = callbacks,\n          class_weight = class_weights\n          )'

In [37]:
'''print("X", x_train.shape, "y", y_train.shape)
model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          shuffle = True,
          verbose=1,
          #validation_split = 0.1,
          callbacks = callbacks,
          class_weight = class_weights
          )'''

'print("X", x_train.shape, "y", y_train.shape)\nmodel.fit(x_train, y_train,\n          validation_data=(x_val, y_val),\n          batch_size=BATCH_SIZE,\n          epochs=NUM_EPOCHS,\n          shuffle = True,\n          verbose=1,\n          #validation_split = 0.1,\n          callbacks = callbacks,\n          class_weight = class_weights\n          )'

In [38]:
'''print("X", x_train.shape, "y", y_train.shape)
model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          shuffle = True,
          verbose=1,
          #validation_split = 0.1,
          callbacks = callbacks,
          class_weight = class_weights
          )'''

'print("X", x_train.shape, "y", y_train.shape)\nmodel.fit(x_train, y_train,\n          validation_data=(x_val, y_val),\n          batch_size=BATCH_SIZE,\n          epochs=NUM_EPOCHS,\n          shuffle = True,\n          verbose=1,\n          #validation_split = 0.1,\n          callbacks = callbacks,\n          class_weight = class_weights\n          )'

In [39]:
print("X", x_train.shape, "y", y_train.shape)
model.fit(x_train, y_train,
          #validation_data=(x_val, y_val),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          shuffle = True,
          verbose=1,
          validation_split = 0.1,
          callbacks = callbacks,
          class_weight = class_weights
          )

X (1099, 400, 400, 3) y (1099, 400, 400, 2)
Train on 989 samples, validate on 110 samples
Epoch 1/5
989/989 [==============================] - 78s 78ms/step - loss: 0.1261 - acc: 0.9470 - val_loss: 0.1799 - val_acc: 0.9260

Epoch 00001: val_acc did not improve from 0.93185
Epoch 2/5
989/989 [==============================] - 78s 78ms/step - loss: 0.1211 - acc: 0.9488 - val_loss: 0.1925 - val_acc: 0.9268

Epoch 00002: val_acc did not improve from 0.93185
Epoch 3/5
989/989 [==============================] - 78s 78ms/step - loss: 0.1170 - acc: 0.9508 - val_loss: 0.2210 - val_acc: 0.9203

Epoch 00003: val_acc did not improve from 0.93185
Epoch 4/5
989/989 [==============================] - 78s 78ms/step - loss: 0.1154 - acc: 0.9515 - val_loss: 0.2010 - val_acc: 0.9241

Epoch 00004: val_acc did not improve from 0.93185
Epoch 5/5
989/989 [==============================] - 77s 78ms/step - loss: 0.1135 - acc: 0.9522 - val_loss: 0.1704 - val_acc: 0.9341

Epoch 00005: val_acc improved from 0.931

In [40]:
print("X", x_train.shape, "y", y_train.shape)
model.fit(x_train, y_train,
          #validation_data=(x_val, y_val),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          shuffle = True,
          verbose=1,
          validation_split = 0.1,
          callbacks = callbacks,
          class_weight = class_weights
          )

X (1099, 400, 400, 3) y (1099, 400, 400, 2)
Train on 989 samples, validate on 110 samples
Epoch 1/5
989/989 [==============================] - 78s 78ms/step - loss: 0.1072 - acc: 0.9545 - val_loss: 0.1703 - val_acc: 0.9321

Epoch 00001: val_acc did not improve from 0.93413
Epoch 2/5
989/989 [==============================] - 78s 78ms/step - loss: 0.1041 - acc: 0.9558 - val_loss: 0.1661 - val_acc: 0.9344

Epoch 00002: val_acc improved from 0.93413 to 0.93435, saving model to /content/drive/My Drive/weightsUnet/weights.best.hdf5
Epoch 3/5
989/989 [==============================] - 78s 78ms/step - loss: 0.1054 - acc: 0.9554 - val_loss: 0.1763 - val_acc: 0.9318

Epoch 00003: val_acc did not improve from 0.93435
Epoch 4/5
989/989 [==============================] - 77s 78ms/step - loss: 0.1019 - acc: 0.9568 - val_loss: 0.1782 - val_acc: 0.9354

Epoch 00004: val_acc improved from 0.93435 to 0.93537, saving model to /content/drive/My Drive/weightsUnet/weights.best.hdf5
Epoch 5/5
989/989 [=====

In [41]:

list_filename = []
prediction_test_dir = "/content/drive/My Drive/predictions_test/"
GT_pred_test_dir = "/content/drive/My Drive/predictions_ground_test/"

if not os.path.isdir(prediction_test_dir):
    os.mkdir(prediction_test_dir)
if not os.path.isdir(GT_pred_test_dir):
    os.mkdir(GT_pred_test_dir)
    
y_submit = np.zeros((((608//IMG_PATCH_SIZE)**2)*TESTING_SIZE,2))
for i in range(1,TESTING_SIZE+1):
  gt_pred, orImg = get_pred_and_ysubmit_pixelwise(test_data_filename, i, 'test', model, PIXEL_DEPTH, NEW_DIM_TRAIN,IMG_PATCH_SIZE,prediction_test_dir)
  gt_filename = GT_pred_test_dir + "gt_pred_" + str(i) + ".png"
  list_filename.append(gt_filename)
  gt_pred.save(gt_filename)
  overlay2 = make_img_overlay_pixel(orImg, gt_pred, PIXEL_DEPTH)
  overlay2.save(GT_pred_test_dir + "overlay_" + str(i) + ".png")
  
  ## FOR OWN VALIDATION ONLY, NEED IT TO COUNT WHITE PATCHES
  gtarr = np.asarray(gt_pred)
  label_patches = img_crop(gtarr, IMG_PATCH_SIZE, IMG_PATCH_SIZE)
  data = np.asarray(label_patches)
  labels = np.asarray([value_to_class(np.mean(data[i])) for i in range(len(data))])
  newPred = label_to_img_unet(gtarr.shape[0], gtarr.shape[1],IMG_PATCH_SIZE, IMG_PATCH_SIZE, gtarr,'test')
  img = Image.fromarray(newPred)
  img.save(prediction_test_dir + "patch_gtimg_" + str(i) + ".png")
  y_submit[((608//IMG_PATCH_SIZE)**2)*(i-1):((608//IMG_PATCH_SIZE)**2)*i,:] = labels
  overlay = make_img_overlay_pixel(orImg, img, PIXEL_DEPTH)
  overlay.save(prediction_test_dir + "overlay_" + str(i) + ".png")
  
  
print('y_submit: ', y_submit.shape)
print('antall vei / antall bakgrunn: ', np.sum(y_submit[:,1]))

y_submit:  (72200, 2)
antall vei / antall bakgrunn:  27588.0


In [42]:
masks_to_submission("kerasMask.csv", *list_filename)


['/content/drive/My Drive/predictions_ground_test/gtimg_1.png', '/content/drive/My Drive/predictions_ground_test/gtimg_2.png', '/content/drive/My Drive/predictions_ground_test/gtimg_3.png', '/content/drive/My Drive/predictions_ground_test/gtimg_4.png', '/content/drive/My Drive/predictions_ground_test/gtimg_5.png', '/content/drive/My Drive/predictions_ground_test/gtimg_6.png', '/content/drive/My Drive/predictions_ground_test/gtimg_7.png', '/content/drive/My Drive/predictions_ground_test/gtimg_8.png', '/content/drive/My Drive/predictions_ground_test/gtimg_9.png', '/content/drive/My Drive/predictions_ground_test/gtimg_10.png', '/content/drive/My Drive/predictions_ground_test/gtimg_11.png', '/content/drive/My Drive/predictions_ground_test/gtimg_12.png', '/content/drive/My Drive/predictions_ground_test/gtimg_13.png', '/content/drive/My Drive/predictions_ground_test/gtimg_14.png', '/content/drive/My Drive/predictions_ground_test/gtimg_15.png', '/content/drive/My Drive/predictions_ground_test

In [0]:
prediction_training_dir = "/content/drive/My Drive/predictions_training/"

if not os.path.isdir(prediction_training_dir):
    os.mkdir(prediction_training_dir)
for i in range(1, TRAINING_SIZE+1):
    oimg, imgpred = get_prediction_with_overlay_pixelwise(train_data_filename, i, 'train', model, PIXEL_DEPTH, NEW_DIM_TRAIN,IMG_PATCH_SIZE)
    oimg.save(prediction_training_dir + "overlay_" + str(i) + ".png")
    imgpred.save(prediction_training_dir + "predictimg_" + str(i) + ".png")


In [44]:
# UNNECESSARY
# Make submission file
prediction_to_submission2('submission_keras.csv', y_submit)

(72200, 2)


**POST-PROCESSING**

In [45]:
new_test_filename = '/content/drive/My Drive/test_set_post_images/'
post_processed_list = []
if not os.path.isdir(new_test_filename):
    os.mkdir(new_test_filename)

y_submit_post = np.zeros((((608//IMG_PATCH_SIZE)**2)*TESTING_SIZE,2))
for i in range(1,TESTING_SIZE+1):
    p_img = get_postprocessed_unet(prediction_test_dir, i, 'test'
    filename = new_test_filename + "processedimg_" + str(i) + ".png"
    post_processed_list.append(filename)
    p_img.save(filename)
    pred = Image.open(filename)
    pred = pred.convert('RGB')
    imageid = "/test_%d" % i
    image_filename = test_data_filename + imageid + imageid + ".png"
    overlay = make_img_overlay_pixel(orImg, pred, PIXEL_DEPTH)
    overlay.save(new_test_filename + "overlay_" + str(i) + ".png")

masks_to_submission("kerasPostprocessedMask.csv", *post_processed_list)

(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)
(608, 608, 3)
(608, 608)


In [46]:
  y_pred_train = np.zeros((((400//IMG_PATCH_SIZE)**2)*TRAINING_SIZE,2))
  y_val_train = np.zeros((((400//IMG_PATCH_SIZE)**2)*TRAINING_SIZE,2))
  for i in range(VALIDATION_SIZE):
    x_img = x_val[i,:,:,:]
    x_img = Image.fromarray(x_img)
    output_prediction = get_prediction_pixel(x_img, model, NEW_DIM_TRAIN) #(1,224,224)
    output_prediction = np.transpose(output_prediction, (1, 2, 0)) #(224,224,1)
    predict_img = np.asarray(output_prediction)

    # Changes into a 3D array, to easier turn into image
    predict_img_3c = np.zeros((predict_img.shape[0],predict_img.shape[1], 3), dtype=np.uint8)
    predict_img8 = np.squeeze(img_float_to_uint8(predict_img, PIXEL_DEPTH))
    predict_img8[predict_img8 >= 100] = 255 
    predict_img8[predict_img8 < 100] = 0        
    predict_img_3c[:,:,0] = predict_img8
    predict_img_3c[:,:,1] = predict_img8
    predict_img_3c[:,:,2] = predict_img8
    #imgpred = Image.fromarray(predict_img_3c)
    label_patches = img_crop(predict_img_3c, IMG_PATCH_SIZE, IMG_PATCH_SIZE)
    data = np.asarray(label_patches)
    labels = np.asarray([value_to_class(np.mean(data[i])) for i in range(len(data))])
    #newPred = label_to_img_unet(gtarr.shape[0], gtarr.shape[1],IMG_PATCH_SIZE, IMG_PATCH_SIZE, gtarr,'test')
    #img = Image.fromarray(newPred)
    #img.save(prediction_test_dir + "patch_gtimg_" + str(i) + ".png")
    y_pred_train[((400//IMG_PATCH_SIZE)**2)*(i):((400//IMG_PATCH_SIZE)**2)*(i+1),:] = labels
    print(labels.shape)
    y_label_patches = img_crop(y_val[i,:,:,:], IMG_PATCH_SIZE, IMG_PATCH_SIZE)
    y_data = np.asarray(y_label_patches)
    y_labels = np.asarray([value_to_class(np.mean(y_data[i])) for i in range(len(y_data))])
    y_val_train[((400//IMG_PATCH_SIZE)**2)*(i):((400//IMG_PATCH_SIZE)**2)*(i+1),:] = y_labels

  
  
tp, tn, fp, fn = f1_values(y_pred_train, y_val_train[:,1])
f1 = f1_measure(tp, fp, fn)
print("f1", f1)

ZeroDivisionError: ignored

In [0]:
for i in range(1,TESTING_SIZE+1):
    #test_data_filename = data_dir + 'test_set_images'
    
    #oimg, gtimg = get_prediction_with_overlay_pixelwise(test_data_filename, i, 'test', model, PIXEL_DEPTH, NEW_DIM_TRAIN)
    #oimg.save(prediction_test_dir + "overlay_" + str(i) + ".png")
    y_submit_post[((608//IMG_PATCH_SIZE)**2)*(i-1):((608//IMG_PATCH_SIZE)**2)*i,:], p_img = get_pred_postprocessed_unet(prediction_test_dir, i, 'test',IMG_PATCH_SIZE)
    filename = new_test_filename + "processedimg_" + str(i) + ".png"
    p_img.save(filename)
    pred = Image.open(filename)
    pred = pred.convert('RGB')
    imageid = "/test_%d" % i
    image_filename = test_data_filename + imageid + imageid + ".png"
    orImg = Image.open(image_filename)
    #img.save(prediction_test_dir + "patch_gtimg_" + str(i) + ".png")
    print(np.asarray(pred).shape)
    print(np.asarray(p_img).shape)
    #y_submit[((608//IMG_PATCH_SIZE)**2)*(i-1):((608//IMG_PATCH_SIZE)**2)*i,:] = labels
    overlay = make_img_overlay_pixel(orImg, pred, PIXEL_DEPTH)
    overlay.save(new_test_filename + "overlay_" + str(i) + ".png")